# Setup Notebook

In [15]:
import IPython
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Data Preparation

In [16]:
msg_df = pd.read_csv('spam.csv', encoding='cp1252')
msg_df = msg_df.drop(msg_df.columns[[2, 3, 4]], axis=1)

In [17]:
msg_df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
# converting label to 0s and 1s 
for index, row in msg_df.iterrows():
        if row["v1"] == "ham":
            row["v1"] = 0
        else:
            row["v1"] = 1

msg_df.head()

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
# split data into feature and labels
label = msg_df['v1']
feature = msg_df['v2']

In [20]:
# split the data into training and testing (30% split)
train_feature, test_feature, train_label, test_label = train_test_split(feature, label, test_size = 0.3)

# uses TF-IDF to identify the features we will be using 
cv = TfidfVectorizer(min_df = 1, stop_words="english")

train_feat_cv = cv.fit_transform(train_feature)
a = train_feat_cv.toarray()

# Create a Model Using Gaussian Naives Bayes

In [21]:
# create a classifier
gnb = GaussianNB()
train_label = train_label.astype('int')

In [22]:
# fit the data
gnb.fit(train_feat_cv.todense(), train_label)

GaussianNB(priors=None)

In [23]:
# identify the features in the testing data
test_feat_cv = cv.transform(test_feature)

In [24]:
# uses the fit mod
pred = gnb.predict(test_feat_cv.todense())

In [25]:
pred

array([0, 1, 0, ..., 0, 1, 1])

In [26]:
actual = np.array(test_label)
type(actual)

numpy.ndarray

In [27]:
type(pred)
count = 0

In [28]:
for i in range(len(pred)): 
    if pred[i] == actual[i]:
        count = count + 1

In [29]:
count

1497

In [30]:
len(pred)

1672

In [31]:
from sklearn.metrics import log_loss
b = test_label.astype('int')

In [32]:
log_loss(b, pred)

3.6150781138208625

In [34]:
from sklearn.metrics import accuracy_score
accuracy_score(b, pred)

0.89533492822966509